# display_svg

> Displaying and writing SVG files from stroke data.

In [ ]:
#| default_exp stroke3.display_svg

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import random


def randcolor(min_color_intensity=0, max_color_intensity=255):
    def _randc():
        return str(random.randint(min_color_intensity, max_color_intensity))

    return f"rgb({_randc()},{_randc()},{_randc()})"

In [ ]:
#| export
import svgwrite


def render_strokes(strokes, target_size=200, stroke_width=1):
    dwg = svgwrite.Drawing(size=(f"{target_size}px", f"{target_size}px"), debug=True)
    the_color = randcolor()

    for points in strokes:
        prev = None
        for row in points:
            x, y = row[0], row[1]
            pt = (x, y)
            if prev:
                # dist = np.linalg.norm(np.array(pt) - np.array(prev))
                # if dist < 60:
                dwg.add(dwg.line(prev, pt, stroke=the_color, stroke_width=stroke_width))
            prev = pt
        the_color = randcolor()
    return dwg

In [ ]:
#| export
def render_deltas(deltas, target_size=200, color_mode=True, stroke_width=1, factor=1.0):
    dwg = svgwrite.Drawing(size=(target_size, target_size))
    dwg.add(
        dwg.rect(
            insert=(0, 0),
            size=(target_size, target_size),
            fill="white",
        )
    )

    lift_pen = 1
    abs_x = 0
    abs_y = 0
    the_color = "#000"
    if color_mode:
        the_color = randcolor()

    for i in range(len(deltas)):
        x = round(float(deltas[i, 0]) * factor, 3)
        y = round(float(deltas[i, 1]) * factor, 3)

        prev_x = round(abs_x, 3)
        prev_y = round(abs_y, 3)

        abs_x += x
        abs_y += y

        if lift_pen == 1:
            p = "M " + str(abs_x) + "," + str(abs_y) + " "
            if color_mode:
                the_color = randcolor()
        else:
            p = (
                "M "
                + str(prev_x)
                + ","
                + str(prev_y)
                + " L "
                + str(abs_x)
                + ","
                + str(abs_y)
                + " "
            )

        lift_pen = deltas[i, 2]

        dwg.add(dwg.path(p).stroke(the_color, stroke_width).fill(the_color))
    return dwg

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()